<a href="https://colab.research.google.com/github/chimera-org/chimera_v2.0/blob/main/notebooks/eegencoder_experiments/02_train_eegencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone your GitHub repository
!git clone https://github.com/chimera-org/chimera_v2.0/

print("✅ Repository cloned.")

Cloning into 'chimera_v2.0'...
remote: Enumerating objects: 1684, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 1684 (delta 169), reused 105 (delta 105), pack-reused 1455 (from 2)
Receiving objects: 100% (1684/1684), 965.12 KiB | 3.66 MiB/s, done.
Resolving deltas: 100% (855/855), done.
✅ Repository cloned.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Setup Environment for EEGEncoder Training

# 1. Install dependencies
!pip install torch torchvision tqdm scikit-learn matplotlib mne numpy scipy --quiet

# 2. Add src to Python path
import sys
import os

SRC_PATH = "/content/chimera_v2.0/src"
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

# 3. Verify path
assert os.path.exists(SRC_PATH), f"Path not found: {SRC_PATH}"

# 4. Import modules
import torch
from eegencoder.model import EEGEncoder
from eegencoder.train import train_model, get_loso_dataloaders, run_loso_cross_validation
from eegencoder.data_loader import BCIC4_2A_Loader
import matplotlib.pyplot as plt

# 5. Verify setup
print("✅ Environment ready")
print(f"📦 PyTorch {torch.__version__}")
print(f"🔥 CUDA: {torch.cuda.is_available()}")
print(f"📁 Path: {SRC_PATH}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 67.2 MB/s eta 0:00:00
✅ Environment ready
📦 PyTorch 2.9.0+cu126
🔥 CUDA: True
📁 Path: /content/chimera_v2.0/src


In [4]:
# Create and Save Preprocessed Data

import os
import numpy as np
from eegencoder.data_loader import BCIC4_2A_Loader

# Path to your extracted GDF files
GDF_PATH = "/content/drive/MyDrive/Motor_Imagery_Datasets/OpenBCI/BCI_cIV_2a/BCI_IV_2a/"

# Where to save processed NPY files
OUTPUT_DIR = "/content/drive/MyDrive/chimera_v2.0/data/bcic4_2a_processed/"

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"📁 Output directory: {OUTPUT_DIR}")

# Initialize loader
loader = BCIC4_2A_Loader(GDF_PATH)

print("="*50)
print("SAVING PREPROCESSED DATA")
print("="*50)

# Save all subjects
for subj_id in range(1, 10):
    print(f"Processing subject {subj_id:02d}...")

    X, y = loader.load_subject(subj_id, training=True)

    np.save(f"{OUTPUT_DIR}subject_{subj_id:02d}_X.npy", X)
    np.save(f"{OUTPUT_DIR}subject_{subj_id:02d}_y.npy", y)

    print(f"  ✅ Saved: X={X.shape}, labels={np.bincount(y)}")

print(f"\n🎉 Done! Data saved to: {OUTPUT_DIR}")

# Verify
files = os.listdir(OUTPUT_DIR)
print(f"Found {len(files)} files: {sorted(files)}")

📁 Output directory: /content/drive/MyDrive/chimera_v2.0/data/bcic4_2a_processed/
SAVING PREPROCESSED DATA
Processing subject 01...
  ✅ Saved: X=(287, 22, 1000), labels=[71 72 72 72]
Processing subject 02...
  ✅ Saved: X=(287, 22, 1000), labels=[72 72 71 72]
Processing subject 03...
  ✅ Saved: X=(287, 22, 1000), labels=[72 72 71 72]
Processing subject 04...
  ✅ Saved: X=(287, 22, 1000), labels=[71 72 72 72]
Processing subject 05...
  ✅ Saved: X=(287, 22, 1000), labels=[72 72 71 72]
Processing subject 06...
  ✅ Saved: X=(287, 22, 1000), labels=[72 72 71 72]
Processing subject 07...
  ✅ Saved: X=(287, 22, 1000), labels=[72 72 71 72]
Processing subject 08...
  ✅ Saved: X=(287, 22, 1000), labels=[72 72 71 72]
Processing subject 09...
  ✅ Saved: X=(287, 22, 1000), labels=[72 72 71 72]

🎉 Done! Data saved to: /content/drive/MyDrive/chimera_v2.0/data/bcic4_2a_processed/
Found 20 files: ['subject_01_X.npy', 'subject_01_train_X.npy', 'subject_01_train_y.npy', 'subject_01_y.npy', 'subject_02_X.np

In [7]:
# CELL: Verify Preprocessed Data Exists
# ======================================
DATA_DIR = "/content/drive/MyDrive/chimera_v2.0/data/bcic4_2a_processed/"

import os

print(f"Checking: {DATA_DIR}")
print("="*50)

if not os.path.exists(DATA_DIR):
    print(f"❌ Directory does NOT exist: {DATA_DIR}")
else:
    files = os.listdir(DATA_DIR)
    npy_files = [f for f in files if f.endswith('.npy')]
    print(f"✅ Found {len(npy_files)} .npy files")

    # Show first few
    print("\nFirst 10 files:")
    for f in sorted(npy_files)[:10]:
        print(f"  - {f}")

    # Check if we have both X and y for subject 01
    x_files = [f for f in npy_files if 'X.npy' in f]
    y_files = [f for f in npy_files if 'y.npy' in f]
    print(f"\nX files: {len(x_files)}, Y files: {len(y_files)}")

# If empty, you need to run the preprocessing cell (Cell 6) in 01_load_bcic4_2a.ipynb

Checking: /content/drive/MyDrive/chimera_v2.0/data/bcic4_2a_processed/
✅ Found 20 .npy files

First 10 files:
  - subject_01_X.npy
  - subject_01_train_X.npy
  - subject_01_train_y.npy
  - subject_01_y.npy
  - subject_02_X.npy
  - subject_02_y.npy
  - subject_03_X.npy
  - subject_03_y.npy
  - subject_04_X.npy
  - subject_04_y.npy

X files: 10, Y files: 10


In [5]:
# Quick Single Subject Test
"""
Before full LOSO, train on one subject to verify everything works
"""

# Data paths
DATA_DIR = "/content/drive/MyDrive/chimera_v2.0/data/bcic4_2a_processed/"
MODEL_SAVE_DIR = "/content/drive/MyDrive/chimera_v2.0/models/"

# Test subject 1 (quick 5 epoch test)
train_loader, test_loader = get_loso_dataloaders(
    DATA_DIR, test_subject=1, batch_size=32
)

print(f"Train batches: {len(train_loader)} | Test batches: {len(test_loader)}")

# Create model
model = EEGEncoder(num_classes=4, pi_compatibility=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Training on device: {device}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

# Quick training
trained_model, history = train_model(
    model, train_loader, test_loader, device,
    epochs=5, lr=1e-3, test_subject=1, save_dir=MODEL_SAVE_DIR
)

# Plot training curve
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot([h['accuracy'] for h in history['train']], label='Train')
plt.plot([h['accuracy'] for h in history['val']], label='Val')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot([h['loss'] for h in history['train']], label='Train')
plt.plot([h['loss'] for h in history['val']], label='Val')
plt.title('Loss')
plt.legend()
plt.show()

Loaded: Train=(2296, 22, 1000), Test=(287, 22, 1000)
Train batches: 72 | Test batches: 9


TypeError: EEGEncoder.__init__() got an unexpected keyword argument 'pi_compatibility'